In [197]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
#query2 = "Die deutsche Liga ist supi!"

def getAllPlayerNames():
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute('SELECT player_name FROM player')
    playerNames = c.fetchall()
    conn.close()
    return playerNames

def getAllLeagues():
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    c.execute('SELECT name FROM league')
    leagueNames = c.fetchall()
    conn.close()
    return leagueNames


In [198]:
#needed to handle "thomas mueller" and "mueller thomas"
def getReverseName(name):
    x = name.find(" ")
    if x > -1:
        return name[x+1:] + " "+ name[:x]
    return "XXX"

#returns the database id of the player by name
def getPlayerIDByName(name):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    queryText = 'SELECT id FROM player WHERE player_name = "'+ name+'"'
    c.execute(queryText)
    playerID = c.fetchall()
    conn.close()
    
    return playerID[0][0]

#Returns list with names of found players
def getPlayersInQuery(query):
    playerList = list()
    for name in getAllPlayerNames():
        if  name[0].find(" ") > -1 and name[0] in query or getReverseName(name[0]) in query:
            playerList.append(name[0])
    return playerList



In [188]:
#needed to handle input like e.g. "1. Bundesliga" instead of "Germany 1. Bundesliga"
def getLeagueNameParts(name):
    wordsGivenLeague = list()
    #do not ignore country-info for england and scotland because their league has the same name
    if "England" not in name and "Scotland" not in name:
        wordsGivenLeague= name.split()
        wordsGivenLeague.pop(0)
    #merge list elements with single characters
    for element in wordsGivenLeague:
        if len(element) < 3:
            index = wordsGivenLeague.index(element)
            if index < 1:
                wordsGivenLeague[0] = element + wordsGivenLeague[1]
            else:
                wordsGivenLeague[1] = wordsGivenLeague[1] + element
    #remove words used in more league-names
    ambiguiteWords = ['League', 'Liga', 'LIGA']
    for word in ambiguiteWords:
        if word in wordsGivenLeague:
            wordsGivenLeague.remove(word)
    return wordsGivenLeague

#returns the database id of the league by name
def getLeagueIDByName(name):
    conn = sqlite3.connect(sqlite_file)
    c = conn.cursor()
    queryText = 'SELECT id FROM league WHERE name = "'+ name+'"'
    c.execute(queryText)
    leagueID = c.fetchall()
    conn.close()
    return leagueID[0][0]

#Returns list with names of found leagues
def getLeaguesInQuery(query):
    leagueList = list()
    for name in getAllLeagues():
        if name[0] in query:
            leagueList.append(name[0])
        else:
            for part in getLeagueNameParts(name[0]):
                if part in query:
                    leagueList.append(name[0])
    return leagueList


In [199]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    print(query)
    return query


In [200]:
query1 = preProcessQuery(query1)
query2 = preProcessQuery(query2)

playersInQuery = list()
for element in getPlayersInQuery(query1):
    playersInQuery.append(getPlayerIDByName(element))
    #print(getPlayerIDByName(element))
    #print(type(getPlayerIDByName(element)))

leaguesInQuery = list()
for element in getLeaguesInQuery(query2):
    leaguesInQuery.append(getLeagueIDByName(element))

print(playersInQuery)
print(leaguesInQuery)

Mueller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!
Bundesliga und Jupiler League sind supi!
[3, 10243]
[1, 7809]
